Primero corremos un script para el build de las librerias de c++ como modulos de python.

In [ ]:
!sh build.sh

Importamos librerias necesarias

In [ ]:
import metnum
import pandas as pd
import numpy as np 
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
import time
from pathlib import Path
from utils import get_MNIST
import matplotlib.pyplot as plt

Obtenemos el dataset de mnist, partido en 4/5 de entrenamiento y 1/5 de validacion. El mismo esta shuffleado sobre si mismo.

In [ ]:
X_train, y_train, X_val, y_val = get_MNIST(0.8)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")

Seteamos la cantidad de alphas a testear (siempre comenzando de 1)

In [71]:
alphas_a_testear = 20
k = 3

Ejecutamos el loop con k para cada alpha, guardamos los resultados en accuracy_con_pca y performance

In [ ]:
accuracy_con_pca = []
performance = []
for i in range(1, alphas_a_testear):
    start =  time.process_time()

    pca = metnum.PCA(i)
    pca.fit(X_train)
    X_train_PCA = pca.transform(X_train)

    X_val_PCA = pca.transform(X_val)

    clf_metnum = metnum.KNNClassifier(k)
    clf_metnum.fit(X_train_PCA, y_train)
    acc = accuracy_score(clf_metnum.predict(X_val_PCA), y_val)
    end = time.process_time()

    time_performance = end - start
    accuracy_con_pca.append(acc)
    performance.append(time_performance)

In [ ]:
accuracy_con_pca

In [ ]:
performance

Generamos un grafico mostrando el ratio accuracy/performance por cada alpha

In [ ]:
plt.plot(np.array(accuracy_con_pca) / np.array(performance), '-o')
plt.xlabel('Alpha')
plt.ylabel('Accuracy / Performance(segundos) ')
plt.xticks(range(1, alphas_a_testear))
plt.title(f'Accuracy / Performance de kNN + PCA para k = {k} y alpha hasta {alphas_a_testear}')
plt.show()

Generamos un grafico. Arriba, tenemos accuracy por cada alpha y abajo su performance.

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(range(1, alphas_a_testear), accuracy_con_pca, '-o')
ax2.plot(range(1, alphas_a_testear), performance, '-o')
ax1.set_title(f'Accuracy y performance de kNN + PCA para k = {k} y alpha de 1 a {alphas_a_testear}')
ax2.set_xlabel('Alpha')
ax2.set_ylabel('Performance (segundos)')
ax1.set_ylabel('Accuracy')
plt.show()